In [4]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import approx_fprime

# Problem Specification

Refer to Paper " Real-time optimization meets Bayesian optimization and derivative-free optimization: A tale of modifier adaptation" Section 4 Case Studies Sub section 4.1 Williams-Otto benchmark problem

## 1. Static Plant

A plant undergoes reactions:
- There are 3 reactions
- A,B are the reactants
- C is a intermediate component
- P is a desired product
- E,G are the by-products

In [9]:
def cstr_p(x,t,u):

    #__Inputs__
    F_B = u[0] # Flowrate of component B
    T_r = u[1] # Temperature of the reactor

    #__States__
    C_A = x[0] # Concentration of component A
    C_B = x[1]
    C_C = x[2]
    C_P = x[3]
    C_E = x[4]
    C_G = x[5]

    #__Process Parameters__
    Ea1 = 6666.7
    Ea2 = 8333.3
    Ea3 = 11111
    F_A = 1.8275
    np1 = 1.6599 * 10e6 # rate constant coefficient
    np2 = 7.2117 * 10e8
    np3 = 2.6742 * 10e12

    #__Equations__
    k1 = np1*np.exp(-Ea1/(T_r+273.15))
    k2 = np2*np.exp(-Ea2/(T_r+273.15))
    k3 = np3*np.exp(-Ea3/(T_r+273.15))

    r1 = k1*C_A*C_B
    r2 = k2*C_B*C_C
    r3 = k3*C_P*C_C

    dC_Adt = -r1
    dC_Bdt = -r1-r2
    dC_Cdt = r1-r2-r3
    dC_Pdt = r2-r3
    dC_Edt = r2
    dC_Gdt = r3

    #__xdot__
    xdot = np.zeros(6)
    xdot[0] = dC_Adt
    xdot[1] = dC_Bdt
    xdot[2] = dC_Cdt
    xdot[3] = dC_Pdt
    xdot[4] = dC_Edt
    xdot[5] = dC_Gdt

    return xdot

## 2. Plant Model

The model have similar but not the same model:
- There are 3 reactions
- A,B are reactants
- P is product 
- G is a byproduct
- There is no intermidiate component C

The model has parameters:
- n is a model parameter for reaction rate constant coefficient
- v is a model parameter for activation energy

In [ ]:
def cstr_m(c,t,u,theta):

    #__Inputs__
    F_B = u[0] # Flowrate of component B
    T_r = u[1] # Temperature of the reactor

    #__States__
    C_A = c[0] # Concentration of component A
    C_B = c[1]
    C_P = c[2]
    C_E = c[3]
    C_G = c[4]

    #__Process Parameters__
    n1 = theta[0]
    n2 = theta[1]
    v1 = theta[2]
    v2 = theta[3]

    #__Equations__
    k1 = n1*np.exp(-v1/(T_r+273.15))
    k2 = n2*np.exp(-v2/(T_r+273.15))

    r1 = k1*C_A*C_B^2
    r2 = k2*C_B*C_B*C_P

    dC_Adt = -r1-r2
    dC_Bdt = -2*r1-r2
    dC_Pdt = r1-r2
    dC_Edt = r1
    dC_Gdt = r2

    #__xdot__
    cdot = np.zeros(5)
    cdot[0] = dC_Adt
    cdot[1] = dC_Bdt
    cdot[2] = dC_Pdt
    cdot[3] = dC_Edt
    cdot[4] = dC_Gdt

    return cdot

## 3. Optimization Problem


In [ ]:
def cost_function(u,t,theta):
       F_A = 1.8275
       F_B = u[0]
       T_r = u[1]
       c = odeint(cstr_m,u,t,args=(theta))
       xp = c[2]/np.sum(c)
       xe = c[3]/np.sum(c)
       return (1043.38*xp + 20.92*xe) * (F_A*F_B) - 79.23*F_A - 118.34*F_B

def cost_function(x0,t,u,theta):
       
       return 1

def ODE_Solver(, h_initial, V_in, t):

       "use odeint from scipy to solve differential eq for time after 1 seconds"

       t = np.linspace(t, t+1, 2)
       # uncertainty = np.random.normal(0,0.5,1)
       uncertainty = 0
       h = odeint(self.differential_eq, h_initial, t, args = (V_in, uncertainty))[1]
       h = np.round(h, decimals = 1)

       return h

# Model-parameter Adaptation

